In [21]:
import os, sys
parent_dir = os.path.abspath('..')
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

In [24]:
from service.InfluxService import InfluxdbService

In [1]:
import pyecharts.options as opts
from pyecharts.charts import Line

In [2]:
week_name_list = ["周一", "周二", "周三", "周四", "周五", "周六", "周日"]
high_temperature = [11, 11, 15, 13, 12, 13, 10]
low_temperature = [1, -2, 2, 5, 3, 2, 0]

In [3]:
line = (
    Line()
    .add_xaxis(xaxis_data=week_name_list)
    .add_yaxis(
        "最高",
        y_axis=high_temperature,
        markline_opts=opts.MarkLineOpts(
            data=[opts.MarkLineItem(type_="average", name="平均值")]
        ),
        color="#1A58A7",
    )
    .set_series_opts(label_opts=opts.LabelOpts(is_show=False),)

)

In [4]:
line.render_notebook()

In [5]:
from pyecharts.faker import Faker
from pyecharts import options as opts
from pyecharts.charts import EffectScatter
from pyecharts.globals import SymbolType


def effectscatter_base() -> EffectScatter:
    c = (
        EffectScatter()
        .add_xaxis(Faker.choose())
        .add_yaxis("", Faker.values())
        .set_global_opts(title_opts=opts.TitleOpts(title="EffectScatter-基本示例"))
    )
    return c


effectscatter_base().render_notebook()